In [1]:
"""
video_illumination_correction.ipynb

Illumination correction pipeline for longitudinal multimodal poultry monitoring dataset.

Applies brightness, contrast, and hue adjustments to raw MP4 video clips recorded
under variable barn lighting conditions. Parameters were calibrated using CapCut
for visual inspection and then implemented programmatically using OpenCV for
automated batch processing.

Input folder structure expected:
    Sample_Dataset/
    └── Video/
          └── *.mp4

Output files are saved in:
    Sample_Dataset/
    └── Video_Processed/
          └── *_processed.mp4

Parameters (fixed as per manuscript, Section 4.3):
    - Brightness:  +25% via linear scaling (alpha = 1.25, beta = 0)
    - Contrast:    +10% (applied via HSV value channel scaling)
    - Hue:         +20% (applied via HSV hue channel shift)

Dependencies:
    pip install opencv-python tqdm numpy
"""

import cv2
import os
import numpy as np
from tqdm import tqdm

In [2]:
# =============================================================================
# PARAMETERS — fixed values as reported in manuscript (Section 4.3)
# =============================================================================

BRIGHTNESS_ALPHA = 1.55
BRIGHTNESS_BETA  = 0
CONTRAST_SCALE   = 1.20
HUE_SHIFT        = -2 # minor correction to neutralise warm barn lighting

INPUT_DIR  = os.path.join("Sample_Dataset", "Video")
OUTPUT_DIR = os.path.join("Sample_Dataset", "Video_Processed")

In [3]:
# =============================================================================
# FRAME-LEVEL CORRECTION
# =============================================================================

def apply_illumination_correction(frame):
    # Brightness: linear scaling across all channels
    frame_bright = cv2.convertScaleAbs(frame, alpha=BRIGHTNESS_ALPHA, beta=BRIGHTNESS_BETA)

    # Convert to HSV for independent channel manipulation
    hsv = cv2.cvtColor(frame_bright, cv2.COLOR_BGR2HSV).astype(np.float32)
    h, s, v = cv2.split(hsv)

    # Contrast: scale Value channel, clip to [0, 255]
    v = np.clip(v * CONTRAST_SCALE, 0, 255)

    # Hue: shift Hue channel, wrap at 180 (OpenCV hue range)
    h = (h + HUE_SHIFT) % 180

    # Merge and convert back to BGR
    hsv_corrected   = cv2.merge([h, s, v]).astype(np.uint8)
    frame_corrected = cv2.cvtColor(hsv_corrected, cv2.COLOR_HSV2BGR)

    return frame_corrected

In [4]:
# =============================================================================
# SINGLE VIDEO PROCESSING
# =============================================================================

def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"  [WARNING] Could not open: {input_path}. Skipping.")
        return

    fps          = cap.get(cv2.CAP_PROP_FPS)
    width        = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height       = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fourcc       = cv2.VideoWriter_fourcc(*"mp4v")
    out          = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    with tqdm(total=total_frames, desc="  Correcting frames", unit="frame", leave=False) as pbar:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            out.write(apply_illumination_correction(frame))
            pbar.update(1)

    cap.release()
    out.release()

In [5]:
# =============================================================================
# BATCH PIPELINE
# =============================================================================

def run_batch_pipeline(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    video_files = [f for f in os.listdir(input_dir) if f.lower().endswith(".mp4")]

    if not video_files:
        print(f"[INFO] No .mp4 files found in: {input_dir}")
        return

    print(f"\n{'='*60}")
    print(f"  Video Illumination Correction Pipeline")
    print(f"  Input  : {input_dir}")
    print(f"  Output : {output_dir}")
    print(f"  Files  : {len(video_files)}")
    print(f"{'='*60}\n")

    for idx, filename in enumerate(tqdm(video_files, desc="Overall progress", unit="video"), start=1):
        input_path  = os.path.join(input_dir, filename)
        base_name   = os.path.splitext(filename)[0]
        output_path = os.path.join(output_dir, f"{base_name}_processed.mp4")

        print(f"[{idx}/{len(video_files)}] {filename}")
        process_video(input_path, output_path)
        print(f"  Saved → {base_name}_processed.mp4\n")

    print(f"{'='*60}")
    print(f"  Done. {len(video_files)} video(s) processed.")
    print(f"  Output saved to: {output_dir}")
    print(f"{'='*60}\n")

In [6]:
# =============================================================================
# RUN
# =============================================================================

run_batch_pipeline(INPUT_DIR, OUTPUT_DIR)


  Video Illumination Correction Pipeline
  Input  : Sample_Dataset/Video
  Output : Sample_Dataset/Video_Processed
  Files  : 2



Overall progress:   0%|          | 0/2 [00:00<?, ?video/s]

[1/2] Barn1_SampleClip_5Sept.mp4


Overall progress:  50%|█████     | 1/2 [00:39<00:39, 39.58s/video]

  Saved → Barn1_SampleClip_5Sept_processed.mp4

[2/2] Barn3_SampleClip_25Aug.mp4


Overall progress: 100%|██████████| 2/2 [01:18<00:00, 39.05s/video]

  Saved → Barn3_SampleClip_25Aug_processed.mp4

  Done. 2 video(s) processed.
  Output saved to: Sample_Dataset/Video_Processed

